In [26]:
import pandas as pd 
import numpy as np
import seaborn as sns

#modeling

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [2]:
data = pd.read_csv('data/stud.csv')

In [3]:
data.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


preparing X and Y variables

In [5]:
X = data.drop(columns=['math_score'],axis=1)

In [6]:
X

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75
...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,99,95
996,male,group C,high school,free/reduced,none,55,55
997,female,group C,high school,free/reduced,completed,71,65
998,female,group D,some college,standard,completed,78,77


In [7]:
y=data['math_score']

In [8]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

Performing Feature engineering 

In [15]:
# create column Transformer with 3 types of transformers

num_features = [feature for feature in X.columns if X[feature].dtype!='O']
cat_features = [feature for feature in X.columns if X[feature].dtype=='O']

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

## we are making a pipeline here as in, this should first do the one hot encoding and the standardisation as one hot encoding will only happen for categoical features 
numeric_transformer = StandardScaler()
one_hot_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", one_hot_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),
    ]
)

X = preprocessor.fit_transform(X)

In [19]:
X.shape

(1000, 19)

In [20]:
# Seperate the dataset into train and test data 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=42)

In [23]:
X_train.shape

(800, 19)

In [24]:
X_test.shape

(200, 19)

Create an evaluate function to give all the metric after model training 

In [25]:
def evaluate_model(true, predicted):
    mse = mean_squared_error(true,predicted)
    mae= mean_absolute_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_score(true, predicted)
    return mse, mae, rmse, r2_square

In [28]:
models = {
    "Linear Regression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K-Neighbors Regressor" : KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest" : RandomForestRegressor(),
    "XGBRegressor" : XGBRegressor(),
    "CatBoostRegressor ": CatBoostRegressor(verbose=False),
    "AdaBoostRegressor": AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train) #train_model

    #make prediction
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #evaluate test and train dataset

    model_train_mse, model_train_mae, model_train_rmse, model_train_r2_score = evaluate_model(y_train,y_train_pred)
    model_test_mse, model_test_mae, model_test_rmse, model_test_r2_score = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for training set')
    print(" - Root mean square error : {: .4f}".format(model_train_rmse))
    print(" - Root mean absolure error : {: .4f}".format(model_train_mae))
    print(" - R2 Score : {: .4f}".format(model_train_mae))

    print('Model performance for testing set')
    print(" - Root mean square error : {: .4f}".format(model_test_rmse))
    print(" - Root mean absolure error : {: .4f}".format(model_test_mae))
    print(" - R2 Score : {: .4f}".format(model_test_mae))

    r2_list.append(model_test_r2_score)


Linear Regression
Model performance for training set
 - Root mean square error :  5.3293
 - Root mean absolure error :  4.2715
 - R2 Score :  4.2715
Model performance for testing set
 - Root mean square error :  5.4252
 - Root mean absolure error :  4.2222
 - R2 Score :  4.2222
Lasso
Model performance for training set
 - Root mean square error :  6.5938
 - Root mean absolure error :  5.2063
 - R2 Score :  5.2063
Model performance for testing set
 - Root mean square error :  6.5197
 - Root mean absolure error :  5.1579
 - R2 Score :  5.1579
Ridge
Model performance for training set
 - Root mean square error :  5.3233
 - Root mean absolure error :  4.2650
 - R2 Score :  4.2650
Model performance for testing set
 - Root mean square error :  5.3904
 - Root mean absolure error :  4.2111
 - R2 Score :  4.2111
K-Neighbors Regressor
Model performance for training set
 - Root mean square error :  5.7079
 - Root mean absolure error :  4.5168
 - R2 Score :  4.5168
Model performance for testing set


In [33]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model Name', 'R2 Score']).sort_values(by='R2 Score',ascending=False)

,Model Name,R2 Score
2,Ridge,0.880593
0,Linear Regression,0.879046
5,Random Forest,0.851973
7,CatBoostRegressor,0.851632
8,AdaBoostRegressor,0.850710
6,XGBRegressor,0.827797
1,Lasso,0.825320
3,K-Neighbors Regressor,0.783813
4,Decision Tree,0.751991


In [41]:
# Accuracy of models 

pd.DataFrame(list(zip(model_list,[r2*100 for r2 in r2_list])),columns=['Model Name', 'Accuracy']).sort_values(by='Accuracy',ascending=False)

,Model Name,Accuracy
2,Ridge,88.059315
0,Linear Regression,87.904648
5,Random Forest,85.197343
7,CatBoostRegressor,85.163189
8,AdaBoostRegressor,85.070964
6,XGBRegressor,82.779657
1,Lasso,82.531973
3,K-Neighbors Regressor,78.381299
4,Decision Tree,75.199147


In [47]:
# Linear regression

lin_model = LinearRegression(fit_intercept=True)
lin_model = lin_model.fit(X_train,y_train)
y_pred = lin_model.predict(X_test)

score = r2_score(y_test, y_pred)*100
print("Accuracy of the model %.2f"%score)

Accuracy of the model 87.90


In [49]:
pred_df= pd.DataFrame({'actual value ':y_test, 'preicted value':y_pred,' difference ': y_pred-y_test})
pred_df

,actual value,preicted value,difference
521,91,76.4375,-14.5625
737,53,59.3125,6.3125
740,80,76.6250,-3.3750
660,74,76.5625,2.5625
411,84,87.7500,3.7500
...,...,...,...
408,52,43.6875,-8.3125
332,62,62.3125,0.3125
208,74,67.7500,-6.2500
613,65,67.0000,2.0000
